## Corpus Linguistics - LING3038/6038
### Week 3

How can we do some descriptive corpus statistics with R?

Two easy ways to get started:

1. a text file that we upload into R
2. text data from the web


In [ ]:
#many packages to install today

install.packages("corpus")

In [ ]:
install.packages("stringr")

In [ ]:
install.packages("tm")

In [ ]:
install.packages(c("ggplot2","stopwords"))

In [ ]:
install.packages("gridExtra")

In [ ]:
install.packages("wordcloud2")

In [ ]:
install.packages("ngram")

First let's look at a text file from the web. There are multiple ways to upload this.

In [ ]:
##string data from a text file or the web
moby<-read.delim("https://www.gutenberg.org/files/2701/2701-0.txt") #this causes some problems!
moby

In [ ]:
moby2<-read.delim("https://www.gutenberg.org/files/2701/2701-0.txt",encoding="UTF-8",sep="\n") #this is better
moby2

In [ ]:
moby3<-read.delim(file.choose(),encoding="UTF-8",sep="\n") #this is a good option for reading a file in from your computer
moby3

In [ ]:
#we can see the first and last bits of our object by using head() and tail()

head(moby2,300)
tail(moby2,300)

In [ ]:
##What do we do with all the header/footer info?
##We can figure out the numbers of the rows where the data starts and ends. 
##Then we can assign that portion to a new object.

moby2[196,]
moby2[197,]

moby2[18928:18929,]

moby.text<-moby2[197:18928,]

In [ ]:
##Here is an alternative way to do the same thing as above.
##We name our lines based on finding certain indicators/signals of when the text starts and stops
##We use the function readLines() from the package {}

moby4<-readLines("https://www.gutenberg.org/files/2701/2701-0.txt",encoding="UTF-8")
head(moby4)

##Then we search for strings that signal the start/stop of the text and name them

# the text starts after the Project Gutenberg header...
start <- grep("START OF THE PROJECT GUTENBERG", moby4, value=F) + 1 #one line after the line with this text

# ...end ends at the Project Gutenberg footer.
stop <- grep("END OF THE PROJECT GUTENBERG", moby4,value=F) - 1 #one line before the line with this text

start
stop



In [ ]:

moby.lines <- moby4[start:stop]
head(moby.lines,20)

In [ ]:
length(moby.text)


In [ ]:
length(moby.lines)

Now let's work with our object 'moby.text'

In [ ]:
head(moby.text)

In [ ]:
moby.text.lowercase<-tolower(moby.text)
head(moby.text.lowercase)

In [ ]:
#Making this object into a vector will be useful for us. Right now it is a XXYY of text strings
is.list(moby.text.lowercase)

moby.text.lowercase.vector<-paste(unlist(moby.text.lowercase), collapse =" ")
head(moby.text.lowercase.vector)

In [ ]:
moby.vec<-moby.text.lowercase.vector #this is just to create a back-up object and work with an object with a shorter name

In [ ]:

library(corpus)
moby.toks.corpus<-text_tokens(moby.vec, drop_punct=T)
moby.toks.corpus

In [ ]:
library(stringr)
moby.toks.stringr<-str_split(moby.vec, "\\s|[[:PUNCT:]]") #this is a regular expression. we'll get to these soon
moby.toks.stringr

In [ ]:
library(tm)
moby.toks.tm.scan<-scan_tokenizer(moby.vec) #this package has a few different tokenizers with different function names
moby.toks.tm.scan